In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
from time import sleep
import csv

# Data Pre-Processing

Zip Code File - Adding Address Data (County, State)

In [2]:
zipcode_df = pd.read_csv('ISyE 6202 2022 Caseworks 3.1 and 3.2 ZIP3 US.csv')

In [2]:
geolocator = Nominatim(user_agent="geoapiExercises")

In [3]:
def georeverse(latitude, longitude):
    
    Latitude = latitude
    Longitude = longitude
    
    location = geolocator.reverse(Latitude+","+Longitude)
    address = location.raw['address']
    
    latlong_to_state = address.get('state', '')
    latlong_to_county = address.get('county','')
    
    return latlong_to_state, latlong_to_county

In [5]:
state_zipdf = []
county_zipdf = []
for i in range(len(zipcode_df)):
    latlong_to_state, latlong_to_county = georeverse(str(zipcode_df.iloc[i]["Lat"]),str(zipcode_df.iloc[i]["Lon"]))
    state_zipdf.append(latlong_to_state)
    county_zipdf.append(latlong_to_county)
    sleep(1)

In [45]:
zipcode_df['State'] = state_zipdf
zipcode_df['County'] = county_zipdf

state_abbreviation_df = pd.read_csv("State Abbreviations.csv")
state_abbreviation_dict = state_abbreviation_df.set_index('State')['Code'].to_dict()

zipcode_df.loc[:,"Abbreviation"] = zipcode_df.loc[:,"State"]
zipcode_df.replace({"Abbreviation":state_abbreviation_dict},inplace=True)

In [46]:
zipcode_df.to_csv('Zip Code - Address Data.csv')

Distribution Center, Fulfillment Center File - Adding Address Data (Latitude, Longitude)

In [10]:
DC_FC_df = pd.read_csv('ISyE 6202 2022 Caseworks 3.1 and 3.2 Distribution and Fulfillment Centers.csv')

In [14]:
def geoforward(postalcode, country):
    
    location = geolocator.geocode(postalcode+","+country)
    
    zip_to_latitude = location.latitude
    zip_to_longitude = location.longitude
    
    return zip_to_latitude, zip_to_longitude

In [15]:
latitude_dcdf = []
longitude_dcdf = []

for i in range(len(DC_FC_df)):
    zip_to_latitude, zip_to_longitude = geoforward(str(DC_FC_df.iloc[i]["ZIP"]),str(DC_FC_df.iloc[i]["Country"]))
    latitude_dcdf.append(zip_to_latitude)
    longitude_dcdf.append(zip_to_longitude)

In [17]:
DC_FC_df['Latitude'] = latitude_dcdf
DC_FC_df['Longitude'] = longitude_dcdf

DC_FC_df.to_csv('DC, FC - Address Data.csv')

Orders File - Adding Product Category, Price, State, Week, Day Columns

In [35]:
orders_df = pd.read_csv('ISyE 6202 2022 Caseworks 3.1 and 3.2 Orders.csv')

orders_df.loc[:,"Price"] = orders_df.loc[:,"Product"]
orders_df.loc[:,"State"] = orders_df.loc[:,"ZIP_3d"]
orders_df.loc[:,"Category"] = orders_df.loc[:,"Product"]
#orders_df.loc[:,"Week of Year"] = orders_df.loc[:,"Order_Date"]
#orders_df.loc[:,"Day of Week"] = orders_df.loc[:,"Order_Date"]

In [36]:
production_categorization_df = pd.read_csv('ISyE 6202 2022 Caseworks 3.1 and 3.2 MetaData-Product Categorization.csv')

production_category_df = production_categorization_df.loc[:,"Product":"Category"]
product_price_df = production_categorization_df.loc[:,["Product","Price per unit (USD)"]]
zip2state_df = zipcode_df.loc[:,["ZIP3","State"]]

product_cat_dict = production_category_df.set_index('Product')['Category'].to_dict()
product_price_dict = product_price_df.set_index('Product')['Price per unit (USD)'].to_dict()
state_dict = zip2state_df.set_index('ZIP3')['State'].to_dict()

In [37]:
orders_df.replace({"Price":product_price_dict, "State": state_dict, "Category":product_cat_dict},inplace=True)

day_of_weekdf = []
week_of_yeardf = []

for i in range(len(orders_df)):
    day_of_weekdf.append(pd.Timestamp(orders_df.iloc[i]["Order_Date"]).day_name())
    week_of_yeardf.append(pd.Timestamp(orders_df.iloc[i]["Order_Date"]).weekofyear)

NameError: name 'day_of_week' is not defined

In [38]:
orders_df["Day of Week"] = day_of_weekdf
orders_df["Week of Year"] = week_of_yeardf

orders_df['Sales'] = orders_df['Price']*orders_df['Quantity']

In [39]:
orders_df.to_csv('Orders - MetaData.csv')